In [53]:
import sqlalchemy as db

In [54]:
# sqlite://<nohostname>/<path>
# where <path> is relative:
engine = db.create_engine("sqlite:///restaurant.db")

In [55]:
connection = engine.connect()
metadata = db.MetaData()


In [56]:


ingredient_table = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(), primary_key=True),
              db.Column('prix_achat', db.Float(), default=0.0),
              )



In [57]:


pays_table = db.Table('Pays', metadata,
              db.Column('pays', db.String(), primary_key=True),
              )



In [58]:


restaurant_table = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(), primary_key=True),
              db.Column('parking', db.Integer(),nullable=False),
              db.Column('espace_enfant', db.Integer(),default=0),
              db.Column('pmr', db.Integer(),default=0),
              db.Column('borne', db.Integer(),default=0),
              db.Column('capacite', db.Integer(),default=0),
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays"),nullable=False),
              )



In [59]:


stock_table = db.Table('Stock', metadata,
              db.Column('id_stock', db.Integer(),primary_key=True),
              db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('quantite', db.Integer(),default=0),
              )



In [60]:


item_table = db.Table('Item', metadata,
              db.Column('nom_item', db.String(),primary_key=True),
              db.Column('type', db.String(),nullable=False),
              db.Column('prix', db.Float(),default=0.00),
              )



In [61]:

carte_item_table = db.Table('CarteItem', metadata,
              db.Column('id_carte_item', db.Integer(),primary_key=True),
              db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [62]:
employes_table= db.Table ('Employes', metadata,
            db.Column('id_employes',db.Integer(),primary_key=True),
            db.Column('nom',db.String(),nullable=False),
            db.Column('role',db.String(),nullable=False),
            db.Column('id_superieur',db.Integer(),db.ForeignKey("Employes.id_employes")),
            db.Column('anciennete',db.Integer(),default=0),
            db.Column('compte_bancaire',db.Integer()),
            db.Column('note',db.Integer()),
            db.Column('adresse',db.String()),
            db.Column('code_postal',db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
            )

In [63]:
bulletin_paye = db.Table('BulletinPaye', metadata,
              db.Column('id_employes', db.Integer(),db.ForeignKey("Employes.id_employes"),primary_key=True),
              db.Column('date', db.String(),primary_key=True,nullable=False),
              db.Column('salaire', db.Float(),nullable=False),
               db.Column('poste', db.String(),nullable=False),
              )

In [64]:
recette = db.Table('Recette', metadata,
        db.Column('id_recette', db.Integer(),primary_key=True),
        db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
        db.Column('quantite_ingredient', db.Integer(),default=0)
        )

In [65]:
menu = db.Table('Menu', metadata,
        db.Column('nom_menu', db.String(),primary_key=True),
        db.Column('plat', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('dessert', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('boisson', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('prix', db.Integer(),default=0.00)
        )

In [66]:
carte_menu_table = db.Table('CarteMenu', metadata,
              db.Column('id_carte_menu', db.Integer(),primary_key=True),
              db.Column('nom_menu', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [67]:
table_addition = db.Table('Addition', metadata,
              db.Column('id_ticket', db.Integer(),primary_key=True),
              db.Column('heure', db.Integer(),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('id_employes', db.String(),db.ForeignKey("Employes.id_employes"),nullable=False),
              db.Column('somme', db.REAL(),default=0),
              db.Column('moyen_paiment', db.String(),nullable=False),
              )

In [68]:
panier_item = db.Table('PanierItem', metadata,
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),primary_key=True),
              db.Column('nom_item', db.Integer(),db.ForeignKey("Item.nom_item"),primary_key=True),
              db.Column('quantite_item', db.Integer(),default=0),
              )

In [69]:
panier_menu = db.Table('PanierMenu', metadata,
              db.Column('id_panier_menu', db.Integer(), primary_key=True),
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),nullable=False),
              db.Column('nom_menu', db.Integer(),db.ForeignKey("Menu.nom_menu"),nullable=False),
              db.Column('quantite_menu', db.Integer(),default=0),
              )

In [70]:
metadata.create_all(engine)